# Using EcoFOCIpy to process raw field data

## DY2107 (GOA PACMAN Cruise - Oscar Dyson)

## CTD / BTL Data

Basic workflow for each instrument grouping is *(initial archive level)*:
- SBE workflow must happen first
- Parse data from btl files into pandas dataframe

Convert to xarray dataframe for all following work *(working or final data level):
- Add metadata from cruise yaml files and/or header info
- ingest metadata from cruise / cast logs
- process data beyond simple file translate
- apply any calibrations or corrections
    + field corrections
    + offsets
    + instrument compensations
    + some QC were available... this would be old-school simple bounds mostly
- adjust time bounds and sample frequency (xarray dataframe)
- save as CF netcdf via xarray: so many of the steps above are optional
    + **ERDDAP NRT** if no corrections, offsets or time bounds are applied but some meta data is
    + **Working and awaiting QC** has no ERDDAP representation and is a holding spot
    + **ERDDAP Final** fully calibrated, qc'd and populated with meta information

Plot for preview and QC
- preview images (indiv and/or collectively)
- manual qc process
- automated qc process ML/AI

Further refinenments for ERDDAP hosting:

## Differences from EPIC standard
- previously btl files had coordinates of lat/lon/time/depth...with a bottle position / fireing order being a variable, this means that if multiple bottles are fired at the same depth, the file was not uniquely indexed and the bottle variable had to be conflated for multiple fireings
- in new version, we will index based on lat/lon/time/bottle_num (bottle number is a sequential unique value... often representing the position on the rosette), merging with CTD downcast data will require maintining a pressure/depth variable in the bottle data that can be rounded to the nearest 1m bin.  This does not solve the problem with multiple discrete samples taken at a single depth and single niskin though.

## Example below is for SBE 9/11+ V2 but the workflow is similar for any SBE instruments.

Future processing of this instrument can be a simplified (no markdown) process which can be archived so that the procedure can be traced or updated

We process each cast as an individual file so this example will not loop over all profiles.  See `example/all_casts.py` example for processing an entire cruise at once.

Adding Discrete samples such as Oxygen, Chlorophyll, Salinity to BTL Data is in `example/discrete_castdata.py`.  Its purpose is to match niskin/bottle information to depth for the discrete data.

In [1]:
import yaml
import glob

import EcoFOCIpy.io.sbe_ctd_parser as sbe_ctd_parser #<- instrument specific
import EcoFOCIpy.io.ncCFsave as ncCFsave
import EcoFOCIpy.metaconfig.load_config as load_config

The sample_data_dir should be included in the github package but may not be included in the pip install of the package

## Simple Processing - first step

In [2]:
sample_data_dir = '/Users/bell/ecoraid/2021/CTDcasts/dy2107/' #root path to cruise directory
ecofocipy_dir = '/Users/bell/Programs/EcoFOCIpy/'

In [3]:
###############################################################
# edit to point to {cruise sepcific} raw datafiles 
datafile = sample_data_dir+'rawconverted/' #<- point to cruise and process all files within
cruise_name = 'DY2107' #no hyphens
cruise_meta_file = sample_data_dir+'logs/DY2107.yaml'
inst_meta_file = sample_data_dir+'logs/FOCI_oneoxy_CTD.yaml' #<- copy to each deployment for simplicity?
group_meta_file = ecofocipy_dir+'staticdata/institutional_meta_example.yaml'
###############################################################

#init and load data
cruise = sbe_ctd_parser.sbe_btl()
filename_list = sorted(glob.glob(datafile + '*.btl'))

cruise_data = cruise.manual_parse(filename_list)

In [4]:
#quick statistical look at the distribution of data for a cast
# #preview a dataframe
cruise_data['ctd001.btl'].describe()

,sbeox0ml/l,sbeox0ps,sbox0mm/kg,sal00,sal11,sigma-t00,sigma-t11,fleco-afl,t090c,t190c,turbwetntu0,prdm,scan
count,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000
mean,3.458329,52.249571,150.642714,33.292571,33.296143,26.072686,26.079671,0.922829,6.488000,6.460529,0.108286,278.568571,57170.000000
std,2.938336,45.954177,128.063803,0.907524,0.902762,1.244304,1.236948,1.417021,3.999312,3.980001,0.127216,368.219425,13190.494797
min,0.305300,4.093000,13.271000,32.212400,32.214300,23.932900,23.933900,0.024400,2.979200,2.979000,0.006800,2.275000,32504.000000
25%,0.814650,11.256500,35.427500,32.436850,32.448300,25.386100,25.411200,0.028800,4.171400,4.171050,0.037250,38.071000,51881.000000
50%,2.874200,40.937000,125.052000,33.566400,33.570800,26.454700,26.455700,0.038100,4.984100,4.975900,0.045200,101.683000,61399.000000
75%,6.345950,96.009000,276.540000,34.013350,34.012000,26.982950,26.981900,1.549150,7.311900,7.219300,0.140150,379.065500,66435.500000
max,6.707600,106.186000,292.241000,34.368800,34.367300,27.383100,27.381900,3.241400,14.486100,14.488100,0.351200,1011.749000,69654.000000


## Time Properties

Not traditionally dealt with for CTD files as they are likely dynamically updated via GPS feed.  However, FOCI tends to label the date/time with the ***at depth*** time-stamp

## Depth Properties and other assumptions

- currently, all processing and binning (1m for FOCI) is done via seabird routines and the windows software.  This may change with the python ctd package for a few tasks

## Add Deployment meta information

In [5]:
#just a dictionary of dictionaries - simple
with open(cruise_meta_file) as file:
    cruise_config = yaml.full_load(file)
cruise_config[cruise_name]

{'CruiseID': 'DY2107',
 'CruiseID_Historic': '',
 'CruiseID_Alternates': '',
 'Project_Leg': '',
 'Vessel': 'R/V Oscar Dyson',
 'ShipID': 'DY',
 'StartDate': datetime.date(2021, 8, 1),
 'EndDate': datetime.date(2021, 8, 25),
 'Project': '',
 'ChiefScientist': '',
 'StartPort': 'Kodiak, AK',
 'EndPort': 'Kodiak, AK',
 'CruiseLocation': 'Gulf of Alaska',
 'Description': 'PACMAN 2021 GOA',
 'CruiseYear': 2021,
 'ctdlogs_pdf_name': 'DY2107_CTDCastLogs.pdf'}

In [6]:
#and if you want a cast from the cruise, just use the consective cast number
cruise_config['CTDCasts']['CTD001']

{'id': 24820,
 'Vessel': 'R/V Oscar Dyson',
 'CruiseID': 'dy2107',
 'Project_Leg': '',
 'UniqueCruiseID': 'dy2107',
 'Project': 'PACMAN 2021 GOA',
 'StationNo_altname': 's1h1',
 'ConsecutiveCastNo': 'CTD001',
 'LatitudeDeg': 57,
 'LatitudeMin': 43.16,
 'LongitudeDeg': 147,
 'LongitudeMin': 48.96,
 'GMTDay': 3,
 'GMTMonth': 'aug',
 'GMTYear': 2021,
 'GMTTime': 18120,
 'DryBulb': 13.5,
 'RelativeHumidity': 86,
 'WetBulb': -99.9,
 'Pressure': 1018,
 'SeaState': '',
 'Visibility': '',
 'WindDir': 43,
 'WindSpd': 9.0,
 'CloudAmt': '',
 'CloudType': '',
 'Weather': '',
 'SurfaceTemp': -99.9,
 'BottomDepth': 3799,
 'StationNameID': '',
 'MaxDepth': 999,
 'InstrumentSerialNos': '',
 'Notes': '',
 'NutrientBtlNiskinNo': '1;2;3;4;5;6;7',
 'NutrientBtlNumbers': '1;2;3;4;5;6;7',
 'OxygenBtlNiskinNo': '',
 'OxygenBtlNumbers': '',
 'SalinityBtlNiskinNo': '1',
 'SalinityBtlNumbers': '167',
 'ChlorophyllBtlNiskinNo': '',
 'ChlorophyllBtlVolumes': '',
 'InstrumentType': 'SBE911plus',
 'WaterMassCode': 

## Add Instrument meta information

Time, depth, lat, lon should be added regardless (always our coordinates) but for a mooring site its going to be a (1,1,1,t) dataset
The variables of interest should be read from the data file and matched to a key for naming.  That key is in the inst_config file seen below and should represent common conversion names in the raw data

In [7]:
with open(inst_meta_file) as file:
    inst_config = yaml.full_load(file)
inst_config

{'time': {'epic_key': 'TIM_601',
  'name': 'time',
  'generic_name': 'time',
  'standard_name': 'time',
  'long_name': 'date and time since reference time'},
 'depth': {'epic_key': 'D_3',
  'generic_name': 'depth',
  'units': 'meter',
  'long_name': 'depth below surface (meters)',
  'standard_name': 'depth',
  'positive': 'down'},
 'latitude': {'epic_key': 'LON_501',
  'name': 'latitude',
  'generic_name': 'latitude',
  'units': 'degrees_north',
  'long_name': 'latitude',
  'standard_name': 'latitude'},
 'longitude': {'epic_key': 'LAT_500',
  'name': 'longitude',
  'generic_name': 'longitude',
  'units': 'degrees_east',
  'long_name': 'longitude',
  'standard_name': 'longitude'},
 'temperature_ch1': {'epic_key': 'T_28',
  'generic_name': 'temp channel 1',
  'long_name': 'Sea temperature in-situ ITS-90 scale',
  'standard_name': 'sea_water_temperature',
  'units': 'degree_C'},
 'temperature_ch2': {'epic_key': 'T2_35',
  'generic_name': 'temp channel 2',
  'long_name': 'Sea temperature i

In [8]:
#sbe data uses header info to name variables... but we want standard names from the dictionary I've created, so we need to rename column variables appropriately
#rename values to appropriate names, if a value isn't in the .yaml file, you can add it

#*** biggest *** difference between moored and profile data is there may be multiple instruments with the same dataype (e.g.) temperature
# on the same platform.  We _used_ to use the phrases primary and secondary, but will now only refer to them as ch1, ch2 etc
cruise_data['ctd001.btl'] = cruise_data['ctd001.btl'].rename(columns={
                        't090c':'temperature_ch1',
                        't190c':'temperature_ch2',
                        'sal00':'salinity_ch1',
                        'sal11':'salinity_ch2',
                        'sbox0mm/kg':'oxy_conc_ch1',
                        'sbeox0ml/l':'oxy_concM_ch1',
                        'sbox1mm/kg':'oxy_conc_ch2',
                        'sbeox1ml/l':'oxy_concM_ch2',
                        'sbeox0ps':'oxy_percentsat_ch1',
                        'sbeox1ps':'oxy_percentsat_ch2',
                        'sigma-t00':'sigma_t_ch1',
                        'sigma-t11':'sigma_t_ch2',
                        'cstarat0':'Attenuation',
                        'cstartr0':'Transmittance',
                        'fleco-afl':'chlor_fluorescence',
                        'turbwetntu0':'turbidity',
                        'empty':'empty', #this will be ignored
                        'prdm':'Pressure [dbar]',
                        'flag':'flag'})

cruise_data['ctd001.btl'].sample()

,oxy_concM_ch1,oxy_percentsat_ch1,oxy_conc_ch1,salinity_ch1,salinity_ch2,sigma_t_ch1,sigma_t_ch2,chlor_fluorescence,temperature_ch1,temperature_ch2,turbidity,Pressure [dbar],scan,datetime
bottle,,,,,,,,,,,,,,
3.0,1.1834,16.421,51.468,33.91,33.9085,26.8642,26.8631,0.0279,4.5219,4.5214,0.0641,252.359,55842.0,2021-08-02 21:18:50


In [9]:
cruise_data['ctd001.btl'].columns

Index(['oxy_concM_ch1', 'oxy_percentsat_ch1', 'oxy_conc_ch1', 'salinity_ch1',
       'salinity_ch2', 'sigma_t_ch1', 'sigma_t_ch2', 'chlor_fluorescence',
       'temperature_ch1', 'temperature_ch2', 'turbidity', 'Pressure [dbar]',
       'scan', 'datetime'],
      dtype='object')

## Add institutional meta-information


In [10]:
with open(group_meta_file) as file:
    group_config = yaml.full_load(file)
group_config

{'source_documents': 'http://www.oceansites.org/docs/oceansites_data_format_reference_manual.pdf',
 'institution': 'Pacific Marine Environmental Lab (PMEL)',
 'project': 'EcoFOCI',
 'project_url': 'https://www.ecofoci.noaa.gov',
 'principal_investigator': 'Phyllis Stabeno',
 'principal_investigator_email': 'phyllis.stabeno (at) noaa.gov',
 'creator_name': 'Shaun Bell',
 'creator_email': 'shaun.bell (at) noaa.gov',
 'creator_institution': 'PMEL',
 'keywords': 'Mooring, Oceanographic',
 'comment': 'Provisional data',
 'sea_area': 'Bering Sea (BS)',
 'featureType': 'timeSeries',
 'conventions': '”CF-1.6, ~OceanSITES-1.5, ACDD-1.2”',
 'license': '',
 'references': '',
 'citation': '',
 'acknowledgement': ''}

In [11]:
# Add meta data and prelim processing based on meta data
# Convert to xarray and add meta information - save as CF netcdf file
# pass -> data, instmeta, depmeta
cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data['ctd001.btl'], 
                                instrument_yaml=inst_config, 
                                operation_yaml=cruise_config,
                                operation_type='ctd')
cruise_data_nc

In [12]:
cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 7)
Coordinates:
  * bottle              (bottle) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0
Data variables: (12/14)
    oxy_concM_ch1       (bottle) float64 0.3053 0.4459 1.183 ... 6.708 6.213
    oxy_percentsat_ch1  (bottle) float64 4.093 6.092 16.42 ... 90.07 101.9 106.2
    oxy_conc_ch1        (bottle) float64 13.27 19.39 51.47 ... 282.1 292.2 271.0
    salinity_ch1        (bottle) float64 34.37 34.12 33.91 ... 32.53 32.35 32.21
    salinity_ch2        (bottle) float64 34.37 34.12 33.91 ... 32.53 32.37 32.21
    sigma_t_ch1         (bottle) float64 27.38 27.1 26.86 ... 25.72 25.05 23.93
    ...                  ...
    temperature_ch1     (bottle) float64 2.979 3.821 4.522 ... 4.984 8.907 14.49
    temperature_ch2     (bottle) float64 2.979 3.821 4.521 ... 4.976 8.702 14.49
    turbidity           (bottle) float64 0.0293 0.0068 0.0641 ... 0.2162 0.3512
    Pressure [dbar]     (bottle) float64 1.012e+03 505.8 252.4 ... 24.91 2.275
    scan                (bottle) float64 3.25e+04 4.792e+04 ... 6.965e+04
    datetime            (bottle) datetime64[ns] 2021-08-02T21:02:37 ... 2021-...

At this point, you could save your file with the `.xarray2netcdf_save()` method and have a functioning dataset.... but it would be very simple with no additional qc, meta-data, or tuned parameters for optimizing software like ferret or erddap.

In [13]:
# expand the dimensions and coordinate variables
# renames them appropriatley and prepares them for meta-filled values
cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

In [14]:
#build list from columsn in data - if a variable isn't in the yaml file, it will be dropped from the final data fields
cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data['ctd001.btl'].columns.values),drop_missing=False)
#adding dimension meta needs to come after updating the dimension values... BUG?
cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])

The following steps can happen in just about any order and are all meta-data driven.  Therefore, they are not required to have a functioning dataset, but they are required to have a well described dataset

In [15]:
cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 7, latitude: 1, longitude: 1, time: 1)
Coordinates:
  * bottle              (bottle) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0
  * latitude            (latitude) float64 1e+35
  * longitude           (longitude) float64 1e+35
  * time                (time) float64 1e+35
Data variables: (12/14)
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 0.3053 .....
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 4.093 ......
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 13.27 ......
    salinity_ch1        (latitude, longitude, time, bottle) float64 34.37 ......
    salinity_ch2        (latitude, longitude, time, bottle) float64 34.37 ......
    sigma_t_ch1         (latitude, longitude, time, bottle) float64 27.38 ......
    ...                  ...
    temperature_ch1     (latitude, longitude, time, bottle) float64 2.979 ......
    temperature_ch2     (latitude, longitude, time, bottle) float64 2.979 ......
    turbidity           (latitude, longitude, time, bottle) float64 0.0293 .....
    Pressure [dbar]     (latitude, longitude, time, bottle) float64 1.012e+03...
    scan                (latitude, longitude, time, bottle) float64 3.25e+04 ...
    datetime            (latitude, longitude, time, bottle) datetime64[ns] 20...

In [16]:
#add global attributes
cruise_data_nc.deployment_meta_add(conscastno='CTD001')

#add instituitonal global attributes
cruise_data_nc.institution_meta_add(group_config)

#add creation date/time - provenance data
cruise_data_nc.provinance_meta_add()

#provide intial qc status field
cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 7, latitude: 1, longitude: 1, time: 1)
Coordinates:
  * bottle              (bottle) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0
  * latitude            (latitude) float64 1e+35
  * longitude           (longitude) float64 1e+35
  * time                (time) float64 1e+35
Data variables: (12/14)
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 0.3053 .....
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 4.093 ......
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 13.27 ......
    salinity_ch1        (latitude, longitude, time, bottle) float64 34.37 ......
    salinity_ch2        (latitude, longitude, time, bottle) float64 34.37 ......
    sigma_t_ch1         (latitude, longitude, time, bottle) float64 27.38 ......
    ...                  ...
    temperature_ch1     (latitude, longitude, time, bottle) float64 2.979 ......
    temperature_ch2     (latitude, longitude, time, bottle) float64 2.979 ......
    turbidity           (latitude, longitude, time, bottle) float64 0.0293 .....
    Pressure [dbar]     (latitude, longitude, time, bottle) float64 1.012e+03...
    scan                (latitude, longitude, time, bottle) float64 3.25e+04 ...
    datetime            (latitude, longitude, time, bottle) datetime64[ns] 20...
Attributes: (12/37)
    CruiseID:                      dy2107
    CruiseID_Historic:             
    CruiseID_Alternates:           
    Project_Leg:                   
    Vessel:                        R/V Oscar Dyson
    ShipID:                        DY
    ...                            ...
    references:                    
    citation:                      
    acknowledgement:               
    date_created:                  2021-11-17T10:09:19Z
    date_modified:                 
    QC_indicator:                  unqcd

## Save CF Netcdf files

Currently stick to netcdf3 classic... but migrating to netcdf4 (default) may be no problems for most modern purposes.  Its easy enough to pass the `format` kwargs through to the netcdf api of xarray.

In [21]:
#loop over all casts and perform tasks shown above

for cast in cruise_data.keys():
    try:
        cruise_data[cast] = cruise_data[cast].rename(columns={
                            't090c':'temperature_ch1',
                            't190c':'temperature_ch2',
                            'sal00':'salinity_ch1',
                            'sal11':'salinity_ch2',
                            'sbox0mm/kg':'oxy_conc_ch1',
                            'sbeox0ml/l':'oxy_concM_ch1',
                            'sbox1mm/kg':'oxy_conc_ch2',
                            'sbeox1ml/l':'oxy_concM_ch2',
                            'sbeox0ps':'oxy_percentsat_ch1',
                            'sbeox1ps':'oxy_percentsat_ch2',
                            'sigma-t00':'sigma_t_ch1',
                            'sigma-t11':'sigma_t_ch2',
                            'cstarat0':'Attenuation',
                            'cstartr0':'Transmittance',
                            'fleco-afl':'chlor_fluorescence',
                            'turbwetntu0':'turbidity',
                            'prdm':'pressure',
                            'empty':'empty', #this will be ignored
                            'flag':'flag'}) #parameters not in list dont get added to file, so no need to remove other than clarity

        cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data[cast], 
                                    instrument_yaml=inst_config, 
                                    operation_yaml=cruise_config,
                                    operation_type='ctd')

        cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

        cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data[cast].columns.values),drop_missing=False)
        #adding dimension meta needs to come after updating the dimension values... BUG?
        cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])
        cruise_data_nc.temporal_geospatioal_meta_data_ctd(positiveE=False,conscastno=cast.split('.')[0])

        #add global attributes cast specific
        cruise_data_nc.deployment_meta_add(conscastno=cast.split('.')[0].upper())

        #add instituitonal global attributes
        cruise_data_nc.institution_meta_add(group_config)

        #add creation date/time - provenance data
        cruise_data_nc.provinance_meta_add()

        #provide intial qc status field
        cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

        cast_lable = cast.lower().split('d')[-1].split('.')[0]
        cruise_data_nc.xarray2netcdf_save(xdf = cruise_data_nc.get_xdf(),
                                   filename=cruise_name+'c'+cast_lable.zfill(3)+'_btl.nc',format="NETCDF3_CLASSIC")
    except:
        print(f'Skipping {cast}')

In [18]:
cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 5, latitude: 1, longitude: 1, time: 1)
Coordinates:
  * bottle              (bottle) float64 1.0 2.0 3.0 4.0 5.0
  * latitude            (latitude) float64 56.3
  * longitude           (longitude) float64 -153.2
  * time                (time) datetime64[ns] 2021-08-25T22:02:00
Data variables: (12/14)
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 1.526 ......
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 21.26 ......
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 66.38 ......
    salinity_ch1        (latitude, longitude, time, bottle) float64 33.84 ......
    salinity_ch2        (latitude, longitude, time, bottle) float64 33.84 ......
    sigma_t_ch1         (latitude, longitude, time, bottle) float64 26.79 ......
    ...                  ...
    temperature_ch1     (latitude, longitude, time, bottle) float64 4.699 ......
    temperature_ch2     (latitude, longitude, time, bottle) float64 4.699 ......
    turbidity           (latitude, longitude, time, bottle) float64 0.262 ......
    pressure            (latitude, longitude, time, bottle) float64 259.2 ......
    scan                (latitude, longitude, time, bottle) float64 8.812e+03...
    datetime            (latitude, longitude, time, bottle) datetime64[ns] 20...

## Next Steps

QC of data (plot parameters with other instruments)
- be sure to updated the qc_status and the history